In [11]:


content = "from\tto\taction\n"

is_header = False
for line in open("./raw_data/cazy_gpt3_extract.tsv", 'r'):
    if is_header == False:
        is_header = True
    else:
        relations = line.strip().split("\t")[2]

        fields = relations.split("|")
        #print (fields)
        subject = fields[0]
        verb = fields[1]
        object = fields[2]

        content += f'{subject}\t{object}\t{verb}\n'

activity_output = open("./raw_data/cazy_subfamily_relation.tsv", 'w')
activity_output.write(content)
activity_output.close()

In [12]:
#lower case the substrate in BINDS
import sys
import pandas as pd
import os

#input_file = sys.argv[1]

input_file = "./raw_data/cazy_subfamily_relation.tsv"

df = pd.read_csv(input_file, sep='\t')

df["doi"] = "CAZy"

df["primary"] = 1

subfiles = df.action.unique()


    #print (df[df.action == s])
    
    #newfile = os.path.join(output_folder, s + ".tsv")

df.loc[df.action == "BINDS", "to"] = df[df.action == "BINDS"]["to"].str.lower()

df.to_csv("./raw_data/cazy_subfamily_relation_lower.tsv", sep="\t", index=False) 

In [13]:
#denormalize the relation file
import sys
import pandas as pd
import os

input_file = "./raw_data/cazy_subfamily_relation_lower.tsv"
output_folder = "./data_for_neo4j_bacteroidetes"

df = pd.read_csv(input_file, sep='\t')

df["doi"] = "CAZy"

df["primary"] = 1

subfiles = df.action.unique()

for s in subfiles:
    #print (df[df.action == s])
    
    newfile = os.path.join(output_folder, s + ".tsv")

    df[df.action == s].to_csv(newfile, sep="\t", index=False) 

In [14]:
import sys
import entity_linkage
import os

#input_file = sys.argv[1]

input_file = "./data_for_neo4j_bacteroidetes/BINDS.tsv"

content = f"from\tto\taction\tdoi\tprimary\n"

is_header = False
for line in open(input_file, 'r'):
    if is_header == False:
        is_header = True
    else:
        fields = line.strip().split("\t")
        if len(fields) == 5:
            sugar = fields[1]
            print (sugar)
            corrected_name = entity_linkage.name_disambiguation(sugar)
            content += f"{fields[0]}\t{corrected_name}\t" + "\t".join(fields[2:]) + "\n"

sugar_output = open("./data_for_neo4j_bacteroidetes/BINDS.tsv", 'w')
sugar_output.write(content)
sugar_output.close()


cellulose
chitin
cellulose
chitin
xylans
cellulose
chitin
xylans
beta-1,3-glucan
beta-1,3-1,4-glucan
beta-1,6-glucan
cellulose
chitin
cellulose
beta-1,3-glucan
beta-1,3-1,4-glucan
beta-1,4-glucan
cellulose
cellulose
xylans
cellulose
beta-1,4-glucan
beta-1,3-1,4-mixed linked glucans
chitin
mannose
xylans
n-acetylgalactosamine
chitin
xylans
xylooligosaccharides
cellulose
glucomannan
cellulose
cellooligosaccharides
chitin
chitin
starch
cyclodextrins
starch
xylans
beta-1,3-glucan
beta-1,4-glucan
mannans
cellulose
alpha-1,3-glucan
starch
starch
mannans
non-crystalline cellulose
cellooligosaccharides
beta-1,3-1,4-glucan
mannans
glucomannan
cellulose
galactose
lactose
polygalacturonic acid
starch
xylans
beta-galactan
xylans
xylans
chitin
inulin
beta-1,3-glucan
lipopolysaccharides
lipoteichoic acid
alpha-glucans
amylose
amylopectin
pullulan
arabinofuranose
beta-1,3-glucan
cellulose
xyloglucan
starch
cellulose
fucose
glycogen
cellulose
chitin
peptidoglycan
galactose
beta-1,3-glucan
starch
xylan

In [15]:
### unify the substrate names from BINDS, DEGRADES and polysaccharide_is_a

import sys
import pandas as pd
import os

#input_file = sys.argv[1]

input_file = "./data_for_neo4j_bacteroidetes/BINDS.tsv"

df = pd.read_csv(input_file, sep='\t')

sugar = df.to.unique()
sugar = set(list(sugar))

In [16]:
import sys
import pandas as pd
import os

#input_file = sys.argv[1]

input_file = "./data_for_neo4j_bacteroidetes/DEGRADES.tsv"

df = pd.read_csv(input_file, sep='\t')

sugar_1 = df.substrate.unique()
sugar_1 = set(list(sugar_1))
sugar = sugar.union(sugar_1)


In [17]:
import sys
import pandas as pd
import os

#input_file = sys.argv[1]

input_file = "./data_for_neo4j_bacteroidetes/polysaccharide_is_a.tsv"

df = pd.read_csv(input_file, sep='\t')

sugar_2 = set(list(df["from"].unique())).union(set(list(df["to"].unique())))
sugar_2 = set(list(sugar_2))
sugar = sugar.union(sugar_2)

In [18]:
content = "name\n" + "\n".join(list(sugar))

sugar_output = open("./data_for_neo4j_bacteroidetes/substrate.tsv", 'w')
sugar_output.write(content)
sugar_output.close()